In [4]:
val sectorDf = spark.read
                    .format("csv")
                    .option("header", true)
                    .option("inferSchema", true)
                    .option("delimitter", ",")
                    .load("hdfs://localhost:9000/stocks/sectors")

sectorDf.printSchema()
sectorDf.show(2)

root
 |-- Company Name: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Series: string (nullable = true)
 |-- ISIN Code: string (nullable = true)

+------------------+------------------+----------+------+------------+
|      Company Name|          Industry|    Symbol|Series|   ISIN Code|
+------------------+------------------+----------+------+------------+
|    Axis Bank Ltd.|FINANCIAL SERVICES|  AXISBANK|    EQ|INE238A01034|
|Bajaj Finance Ltd.|FINANCIAL SERVICES|BAJFINANCE|    EQ|INE296A01024|
+------------------+------------------+----------+------+------------+
only showing top 2 rows



sectorDf: org.apache.spark.sql.DataFrame = [Company Name: string, Industry: string ... 3 more fields]


In [5]:
// Write dataframe into mysql
// overwrite - sectors/existing  table, data shall be deleted
// new table created, records shall be inserted
// "jdbc" - write to connect to db
// driver - mysql class name
// user - mysql user name
// password - mysql password
// dbtable - mysql table name
sectorDf
.withColumnRenamed("Company Name", "Company")
.withColumnRenamed("ISIN Code", "ISIN")
.write
 .mode("overwrite")
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/stockdb")
.option("driver", "com.mysql.jdbc.Driver")
.option("user", "team")
.option("password", "Team1234!")
.option("dbtable", "sectors")
 .save()

In [6]:
// Now read data from mysql to DataFrame
// this will create schema automatically because database
// is structured already, it has table, column, data type etc
// Spark will read scheme definition from mysql and create dataframe
// automatically
val sectorDataFromMySqlDf = spark.read
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/stockdb")
.option("driver", "com.mysql.jdbc.Driver")
.option("user", "team")
.option("password", "Team1234!")
.option("dbtable", "sectors")
 .load()

sectorDataFromMySqlDf.printSchema()
sectorDataFromMySqlDf.show(5)

root
 |-- Company: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Series: string (nullable = true)
 |-- ISIN: string (nullable = true)

+--------------------+------------+----------+------+------------+
|             Company|    Industry|    Symbol|Series|        ISIN|
+--------------------+------------+----------+------+------------+
|Brigade Enterpris...|CONSTRUCTION|   BRIGADE|    EQ|INE791I01019|
|            DLF Ltd.|CONSTRUCTION|       DLF|    EQ|INE271C01023|
|Godrej Properties...|CONSTRUCTION|GODREJPROP|    EQ|INE484J01027|
|Indiabulls Real E...|CONSTRUCTION| IBREALEST|    EQ|INE069I01010|
|Macrotech Develop...|CONSTRUCTION|     LODHA|    EQ|INE670K01029|
+--------------------+------------+----------+------+------------+
only showing top 5 rows



sectorDataFromMySqlDf: org.apache.spark.sql.DataFrame = [Company: string, Industry: string ... 3 more fields]
